In [31]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.utils import column_or_1d
from sklearn.preprocessing import label_binarize
import numpy as np
import nibabel as nib

In [ ]:
img = nib.load('UKCHLL_001_000.nii.gz')

data = img.get_fdata()

print(data.shape) 

(512, 512, 953)


In [22]:
#AUROC
""" y_true=true labels et y_score= score d'incertitude"""
y_true = [1,0,1,1,0,1,0,0,0,1]
y_score = [0.1, 0.4, 0.35, 0.8, 0.7, 0.2, 0.9, 0.6, 0.8, 0.4]

fpr, tpr, _ = roc_curve(y_true, y_score)
roc_auc = auc(fpr, tpr)

print(roc_auc)
#OOD detection rate
""" ils font juste nombre de ood trouvés/nb ood mais la manière dont est défini le OOD est peu claire et semble arbitraire"""

0.16000000000000003


' ils font juste nombre de ood trouvés/nb ood mais la manière dont est défini le OOD est peu claire et semble arbitraire'

In [ ]:
#DICE et IoU
#exemple
y_true = [1, 0, 1, 1, 0, 1, 0]
y_pred = [1, 0, 1, 0, 0, 1, 1]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

dice=2 * tp / (2 * tp + fp + fn)
iou=tp / (tp + fp + fn)

print(dice,iou)

0.75 0.6


In [ ]:
#ACE
#correct=1 si y_true=y_pred et confid=confiance du modèle

def calib_stats(correct, calib_confids):
    # calib_confids = np.clip(self.confids, 0, 1)
    n_bins = 20
    y_true = column_or_1d(correct)
    y_prob = column_or_1d(calib_confids)

    if y_prob.min() < 0 or y_prob.max() > 1:
        raise ValueError(
            "y_prob has values outside [0, 1] and normalize is " "set to False."
        )

    labels = np.unique(y_true)
    if len(labels) > 2:
        raise ValueError(
            "Only binary classification is supported. " f"Provided labels {labels}."
        )
    y_true = label_binarize(y_true, classes=labels)[:, 0]

    bins = np.linspace(0.0, 1.0 + 1e-8, n_bins + 1)

    binids = np.digitize(y_prob, bins) - 1

    bin_sums = np.bincount(binids, weights=y_prob, minlength=len(bins))
    bin_true = np.bincount(binids, weights=y_true, minlength=len(bins))
    bin_total = np.bincount(binids, minlength=len(bins))

    nonzero = bin_total != 0
    num_nonzero = len(nonzero[nonzero == True])
    prob_true = bin_true[nonzero] / bin_total[nonzero]
    prob_pred = bin_sums[nonzero] / bin_total[nonzero]
    prob_total = bin_total[nonzero] / bin_total.sum()

    bin_discrepancies = np.abs(prob_true - prob_pred)
    return bin_discrepancies, prob_total, num_nonzero


def calc_ace(correct, calib_confids):
    bin_discrepancies, _, num_nonzero = calib_stats(correct, calib_confids)
    return (1 / num_nonzero) * np.sum(bin_discrepancies)

#exemple
correct= [0, 1, 0, 1, 0, 1, 1, 0, 0, 1]
calib_confids= [0.1, 0.9, 0.4, 0.8, 0.2, 0.6, 0.7, 0.3, 0.4, 0.9]
calc_ace(correct,calib_confids)

np.float64(0.25)

In [ ]:
#NCC
def compute_ncc(gt_unc_map: np.array, pred_unc_map: np.array):
    """
    Compute the normalized cross correlation between a ground truth uncertainty and a predicted uncertainty map,
    to determine how similar the maps are.
    :param gt_unc_map: the ground truth uncertainty map based on the rater variability
    :param pred_unc_map: the predicted uncertainty map
    :return: float: the normalized cross correlation between gt and predicted uncertainty map
    """
    mu_gt = np.mean(gt_unc_map)
    mu_pred = np.mean(pred_unc_map)
    sigma_gt = np.std(gt_unc_map, ddof=1)
    sigma_pred = np.std(pred_unc_map, ddof=1)
    gt_norm = gt_unc_map - mu_gt
    pred_norm = pred_unc_map - mu_pred
    prod = np.sum(np.multiply(gt_norm, pred_norm))
    ncc = (1 / (np.size(gt_unc_map) * sigma_gt * sigma_pred)) * prod
    return ncc

#exemple
gt_unc_map = np.array([
    [0.2, 0.3, 0.4, 0.5, 0.6],
    [0.3, 0.3, 0.5, 0.6, 0.7],
    [0.4, 0.4, 0.6, 0.7, 0.8],
    [0.5, 0.6, 0.7, 0.8, 0.9],
    [0.6, 0.7, 0.8, 0.9, 1.0]
])

pred_unc_map = np.array([
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.3, 0.4, 0.5, 0.6, 0.7],
    [0.4, 0.5, 0.6, 0.7, 0.8],
    [0.6, 0.7, 0.8, 0.9, 1.0],
    [0.5, 0.6, 0.7, 0.8, 0.9]
])

compute_ncc(gt_unc_map,pred_unc_map)

np.float64(0.8932699455139714)

In [ ]:
#AURC et EAURC
"""
risk=array avec erreurs/loss pour chaque image
confids=confiance dans la préd
"""

def rc_curve_stats(
    risks: np.array, confids: np.array
) -> tuple[list[float], list[float], list[float]]:
    coverages = []
    selective_risks = []
    assert (
        len(risks.shape) == 1 and len(confids.shape) == 1 and len(risks) == len(confids)
    )

    n_samples = len(risks)
    idx_sorted = np.argsort(confids)

    coverage = n_samples
    error_sum = sum(risks[idx_sorted])

    coverages.append(coverage / n_samples)
    selective_risks.append(error_sum / n_samples)

    weights = []

    tmp_weight = 0
    for i in range(0, len(idx_sorted) - 1):
        coverage = coverage - 1
        error_sum = error_sum - risks[idx_sorted[i]]
        tmp_weight += 1
        if i == 0 or confids[idx_sorted[i]] != confids[idx_sorted[i - 1]]:
            coverages.append(coverage / n_samples)
            selective_risks.append(error_sum / (n_samples - 1 - i))
            weights.append(tmp_weight / n_samples)
            tmp_weight = 0
    return coverages, selective_risks, weights

def aurc(risks: np.array, confids: np.array):
    _, risks, weights = rc_curve_stats(risks, confids)
    return sum(
        [(risks[i] + risks[i + 1]) * 0.5 * weights[i] for i in range(len(weights))]
    )

def eaurc(risks: np.array, confids: np.array):
    """Compute normalized AURC, i.e. subtract AURC of optimal CSF (given fixed risks)."""
    n = len(risks)
    # optimal confidence sorts risk. Asencding here because we start from coverage 1/n
    selective_risks = np.sort(risks).cumsum() / np.arange(1, n + 1)
    aurc_opt = selective_risks.sum() / n
    return aurc(risks, confids) - aurc_opt


#exemple
confids = np.array([0.9, 0.8, 0.3, 0.6])  

risks = np.array([0, 1, 1, 0]) 
print(aurc(risks,confids))
print(eaurc(risks,confids))

0.2708333333333333
0.0625


In [ ]:
#UNC MAP (mais très emmêlé) --> utile ?
def get_gt_unc_map(self, image_id):
        if self.exp_version.gt_unc_map_loading is None:
            n_reference_segs = self.exp_version.n_reference_segs
            reference_segs_paths = [
                self.ref_seg_dir / f"{image_id}_{i:02d}{self.exp_version.image_ending}"
                for i in range(n_reference_segs)
            ]
            reference_segs = []
            for reference_seg_path in reference_segs_paths:
                reference_seg, _ = load(reference_seg_path)
                reference_segs.append(reference_seg)
            reference_segs = np.array(reference_segs)
            per_pixel_variance = np.var(reference_segs, axis=0)
        else:
            per_pixel_variance = hydra.utils.instantiate(
                self.exp_version.gt_unc_map_loading,
                image_id=image_id,
                dataloader=self.dataloader,
            )
        return per_pixel_variance